In [1]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Load demand data
load_df = pd.read_csv('ieso_ga_master_dataset_allWeather_updated2020.csv')
load_df.rename(columns={'timestamp_eastern': 'ts'}, inplace=True)
load_df['ts'] = pd.to_datetime(load_df['ts'])
# load_df.head(48)

In [3]:
# Load forecasts
forecast2_df = pd.read_csv('ga_forecasts_top_2.csv', index_col=0)
forecast2_df['ts'] = pd.to_datetime(forecast2_df['ts'])
forecast2_df['ts_future'] = pd.to_datetime(forecast2_df['ts_future'])
# forecast2_df.head(48)

In [4]:
# Look at the forecasts at 10:00 am on each day
mask = forecast2_df.ts.dt.time == datetime.time(10, 0)
forecast2_df = forecast2_df[mask]
forecast2_df

,forecast,ts,ts_future,horizon_hr
216,0.0,2016-05-08 10:00:00,2016-05-08 11:00:00,1.0
217,0.0,2016-05-08 10:00:00,2016-05-08 12:00:00,2.0
218,0.0,2016-05-08 10:00:00,2016-05-08 13:00:00,3.0
219,0.0,2016-05-08 10:00:00,2016-05-08 14:00:00,4.0
220,0.0,2016-05-08 10:00:00,2016-05-08 15:00:00,5.0
...,...,...,...,...
171331,0.0,2020-02-29 10:00:00,2020-03-01 06:00:00,20.0
171332,0.0,2020-02-29 10:00:00,2020-03-01 07:00:00,21.0
171333,0.0,2020-02-29 10:00:00,2020-03-01 08:00:00,22.0
171334,0.0,2020-02-29 10:00:00,2020-03-01 09:00:00,23.0


In [5]:
# Find out the highest peak for each day in a season
load_df['rankings_per_day'] = load_df.groupby(['season', load_df.ts.dt.date]).adjusted_demand_MW.rank(ascending=False)
# load_df.head(48)


In [6]:
# Filter out the max value in each day and store as peaks_df
mask = load_df['rankings_per_day']==1.0
peaks_df = load_df[mask]
# peaks_df

In [7]:
# Filter out the top 1, 5, 10, 20 peaks in each season
# Reset index
peaks_df.reset_index(drop=True, inplace=True)

# Group by season and rank by adjusted_demand_MW
peaks_df['rankings_per_season'] = peaks_df.groupby(['season']).adjusted_demand_MW.rank(ascending=False)
peaks_df.head(10)

/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,adjusted_demand_MW,demand_MW,hour_ending_eastern,season,ts,timestamp_utc,altitude_Toronto,apparentTemperature_Toronto,azimuth_Toronto,cloudCover_Toronto,...,summary_Milton,temperature_Milton,uvIndex_Milton,visibility_Milton,windBearing_Milton,windGust_Milton,windSpeed_Milton,population_Milton,rankings_per_day,rankings_per_season
0,16100.0,16206.0,20,2013-2014,2013-05-01 20:00:00,2013-05-02 01:00:00,0.0,18.33,0.0,0.07,...,Clear,14.35,0.0,16.093,89.0,1.90,1.84,101715.0,1.0,303.0
1,16032.0,16460.0,21,2013-2014,2013-05-02 21:00:00,2013-05-03 02:00:00,0.0,16.82,0.0,0.02,...,Clear,13.06,0.0,16.093,72.0,2.47,2.04,101715.0,1.0,308.0
2,15428.0,15780.0,20,2013-2014,2013-05-03 20:00:00,2013-05-04 01:00:00,0.0,17.80,0.0,0.02,...,Clear,13.94,0.0,16.093,58.0,4.95,2.46,101715.0,1.0,335.0
3,14191.0,14473.0,21,2013-2014,2013-05-04 21:00:00,2013-05-05 02:00:00,0.0,16.43,0.0,0.01,...,Clear,11.99,0.0,16.093,54.0,2.02,1.22,101715.0,1.0,362.0
4,14713.0,14944.0,21,2013-2014,2013-05-05 21:00:00,2013-05-06 02:00:00,0.0,18.52,0.0,0.30,...,Clear,13.08,0.0,16.093,68.0,1.39,1.39,101715.0,1.0,352.0
5,16157.0,16481.0,20,2013-2014,2013-05-06 20:00:00,2013-05-07 01:00:00,0.0,15.81,0.0,0.05,...,Partly Cloudy,13.43,0.0,16.093,65.0,1.17,1.01,101715.0,1.0,300.0
6,16275.0,16648.0,21,2013-2014,2013-05-07 21:00:00,2013-05-08 02:00:00,0.0,18.49,0.0,0.47,...,Partly Cloudy,14.89,0.0,16.093,52.0,0.93,0.90,101715.0,1.0,295.0
7,16324.0,16694.0,20,2013-2014,2013-05-08 20:00:00,2013-05-09 01:00:00,0.0,16.84,0.0,0.98,...,Mostly Cloudy,14.06,0.0,15.875,81.0,1.10,0.88,101715.0,1.0,291.0
8,16239.0,16611.0,21,2013-2014,2013-05-09 21:00:00,2013-05-10 02:00:00,0.0,17.04,0.0,0.02,...,Clear,16.97,0.0,16.093,261.0,1.60,1.60,101715.0,1.0,297.0
9,15681.0,16032.0,20,2013-2014,2013-05-10 20:00:00,2013-05-11 01:00:00,0.0,13.28,0.0,1.00,...,Overcast,13.74,0.0,14.903,214.0,3.91,1.79,101715.0,1.0,325.0


In [10]:
forecast2_results = forecast2_df[['ts_future','forecast']]
forecast2_results = forecast2_results.rename(columns={'ts_future':'ts'})

keys1 = ['season', 'top n', 'success']
keys2 = ['12', '16', '17', '18', '19', '20']
keys = keys1+keys2
d = {}
for k in keys:
    d[k] = []
# print('dictionary:', d)

for n in [1, 5, 10, 20]:
    # Select rows with 'rankings_per_season' <= n: top n df
    mask_top_n = peaks_df['rankings_per_season'] <= n
    top_n_df = peaks_df[mask_top_n]

    # Merge top n df with forecasting df
    top_n_results = top_n_df[['adjusted_demand_MW', 'demand_MW', 'season','ts','rankings_per_day','rankings_per_season']]
    top_n_results = pd.merge(top_n_results, forecast2_results, on='ts')
    
    # Generate series with number of successes for each season
    top_n_successes = top_n_results.groupby('season').forecast.apply(lambda x: (x > 0).sum()).rename('successes')
    # Get the index names, i.e., the list of seasons
    season_list = top_n_successes.index.values

    # For each season, append results to dictionary
    for row_index in season_list:
        # Append season name
        d['season'].append(row_index)
        # Append hit rate (converted to %)
        d['success'].append(int(top_n_successes.loc[row_index]/n*100))
        # Append top n
        d['top n'].append(n)


        # Select rows for current season
        mask = top_n_results.season == row_index
        season_results = top_n_results[mask]
        
        # For each hour in keys, count the times that this hour is peak hour and 
        # the times that the forecasts at this hour is larger than zero
        for k in keys2:
            # Select rows for current hour
            mask_k = season_results.ts.dt.time == datetime.time(int(k),0)
            season_results_k = season_results[mask_k]
            
            # Count the times that that the forecasts at this hour is larger than zero
            num_hit_k = season_results_k.forecast.gt(0).sum()
            # Count the times that this hour is peak hour in current season
            total_num_k = season_results_k.shape[0]

            # Append num_hits/total_num to dictionary
            d[k].append(str(num_hit_k)+'/'+str(total_num_k))

results_df = pd.DataFrame(data=d)
results_df

,season,top n,success,12,16,17,18,19,20
0,2016-2017,1,100,0/0,0/0,0/0,1/1,0/0,0/0
1,2017-2018,1,100,0/0,0/0,1/1,0/0,0/0,0/0
2,2018-2019,1,100,0/0,0/0,1/1,0/0,0/0,0/0
3,2019-2020,1,100,0/0,0/0,1/1,0/0,0/0,0/0
4,2016-2017,5,100,0/0,0/0,3/3,2/2,0/0,0/0
5,2017-2018,5,100,0/0,0/0,3/3,2/2,0/0,0/0
6,2018-2019,5,80,0/0,0/0,3/3,1/1,0/0,0/0
7,2019-2020,5,80,0/1,0/0,3/3,1/1,0/0,0/0
8,2016-2017,10,90,0/1,0/0,7/7,2/2,0/0,0/0
9,2017-2018,10,100,0/0,0/0,4/4,6/6,0/0,0/0
